In [ ]:
# default_exp factory_defect_statistics

# Factory Defect Statistics

In [ ]:
#export
from nbdev.showdoc import *
import jovsatools
import fastcore
import numpy as np
import scipy as sp
from collections import defaultdict

### Question

There are 2 factories producing tablets: one produces tablets with probability of defect 20%,  another with probability of defect 80%

We have ordered single shipment having 2 tablets and received the entire shipment from the factory chosen evenly.

We opened the first tablet and it has a defect,  what is the probability that the second tablet having defect as well ?

**Assumptions**: 
* Factory defects are independent and can be sampled without replacement

## Statistical Approach

In [ ]:
"""
ans = 0.5 * case 1 + 0.5* case_2

s.t: 
    case 1 [factory 1] = P(exactly 2 defects from factory 1) = P(<=2 defects) - P(<= 1 defect)
    case 2 [factory 2] = P(exactly 2 defects from factory 1) = P(<=2 defects) - P(<= 1 defect)

"""
print()

In [ ]:
case_1 = sp.stats.binom.pmf(k=2, n=2, p=0.2)
print(f"case_1: {case_1}")

case_2 = sp.stats.binom.pmf(k=2, n=2, p=0.8)
print(f"case_2: {case_2}")

ans = 0.5*case_1 + 0.5*case_2
print(f"ans:{ans}")

case_1: 0.04000000000000001
case_2: 0.64
ans:0.34


### Numerical Approach

In [ ]:
class Simulator:
    def __init__(self):
        np.random.seed(1123)
        self.factory_defect_rate = {
            0: 0.2,
            1: 0.8,
        }
        self.stats = defaultdict(int)
    
    def choose_factory(self):
        return np.random.randint(2, size=1)[0]
    
    def is_defect(self, factory):
        defect_rate = self.factory_defect_rate[factory]
        x = np.random.random()
        if x <= defect_rate:
            return True
        else:
            return False
    
    def produce_shipment(self, shipment_size=2):
        factory = self.choose_factory()
        res = []
        for item in range(shipment_size):
            is_defect = self.is_defect(factory)
            res.append(is_defect)
            if is_defect: 
                # (factory, item, is_defective)
                self.stats[(factory, item, 1)] += 1

            else:
                # (factory, item, defective)
                self.stats[(factory, item, 0)] += 1
            
        # case when both tablets are defective
        if res == [True,True]:
            self.stats[(factory, 'match')] += 1
    
    def run(self, trials):
        for trial in range(trials):
            self.produce_shipment()
        self.stats['trials'] = trials

In [ ]:
simulator = Simulator()
simulator.run(trials=10000)
ans = (simulator.stats[(0, 'match')] + simulator.stats[(1, 'match')])/simulator.stats['trials']
print(f"ans:{ans}")

ans:0.3359


### Jovan's Debugging

In [ ]:
simulator.stats

defaultdict(int,
            {(0, 0, 0): 4019,
             (0, 1, 1): 1013,
             (0, 0, 1): 981,
             (0, 1, 0): 3987,
             (1, 0, 1): 3992,
             (1, 1, 1): 3985,
             (1, 'match'): 3160,
             (1, 0, 0): 1008,
             (1, 1, 0): 1015,
             (0, 'match'): 199,
             'trials': 10000})

In [ ]:
"""
{(0, 0, 0): 4074,
 (0, 0, 1): 1013,
 (0, 1, 0): 4070,
 (0, 1, 1): 1017,
 

 (1, 0, 0): 1008,
 (1, 0, 1): 3905,
 (1, 1, 0): 966,
 (1, 1, 1): 3947,

 'trials': 10000})

"""

"\n{(0, 0, 0): 4074,\n (0, 0, 1): 1013,\n (0, 1, 0): 4070,\n (0, 1, 1): 1017,\n \n\n (1, 0, 0): 1008,\n (1, 0, 1): 3905,\n (1, 1, 0): 966,\n (1, 1, 1): 3947,\n\n 'trials': 10000})\n\n"